In [46]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
%autoreload 2

In [48]:
from IPython.display import Image
from base64 import b64decode

import k3d
import randomcolor

In [49]:

def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='3d'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = np.max(samples_distances)
            print(max_dist)
            colors = k3d.helpers.map_colors(
                samples_distances, k3d.colormaps.basic_color_maps.WarmCool, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='3d'
        
        if None is not directions:
            directions_to_plot = np.hstack((samples, samples + directions))
            
            for i, dir_to_plot in enumerate(directions_to_plot):
                dir_to_plot = dir_to_plot.reshape((2, 3))
                if np.all(dir_to_plot[0] == dir_to_plot[1]):
                    continue
                color = int(colors[i]) if None is not colors else samples_color
                plt_line = k3d.line(dir_to_plot, 
                                    shader='mesh', width=directions_width, color=color)
                plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='3d'
        
        if None is not sharp_curves:
            rand_color = randomcolor.RandomColor()
            for i, vert_ind in sharp_curves.items():
                sharp_points_curve = mesh.vertices[vert_ind]
                if None is not sharpcurve_color:
                    color = sharpcurve_color
                else:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [50]:
import sys
sys.path.append('/home/artonson/repos/sharp_features/')

In [51]:

from sharpf.data.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE, PoissonDiskSampler
from sharpf.utils.mesh_utils import trimesh_load

from copy import deepcopy
import json

import yaml
from yaml import Loader
import numpy as np
from tqdm import tqdm

In [52]:
with open('/home/artonson/tmp/00000002/00000002_1ffb81a71e5b402e966b9341_trimesh_001.obj', 'rb') as f:
    mesh = trimesh_load(f)

In [53]:
with open('/home/artonson/tmp/00000002/00000002_1ffb81a71e5b402e966b9341_features_001.yml') as f:
    features = yaml.load(f)

In [54]:
with open('/home/artonson/repos/sharp_features/sharpf/data/dataset_config.json') as f:
    config = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/artonson/repos/sharp_features/sharpf/data/dataset_config.json'

In [55]:


def load_func_from_config(func_dict, config):
    return func_dict[config['type']].from_config(config)


def compute_curves_nbhood(features, vert_indices, face_indexes):
    """Extracts curves for the neighbourhood."""
    nbhood_sharp_curves = []
    for curve in features['curves']:
        nbhood_vert_indices = np.array([
            vert_index for vert_index in curve['vert_indices']
            if vert_index + 1 in vert_indices
        ])
        if len(nbhood_vert_indices) == 0:
            continue
        for index, reindex in zip(vert_indices, np.arange(len(vert_indices))):
            nbhood_vert_indices[np.where(nbhood_vert_indices == index - 1)] = reindex
        nbhood_curve = deepcopy(curve)
        nbhood_curve['vert_indices'] = nbhood_vert_indices
        nbhood_sharp_curves.append(nbhood_curve)

    nbhood_features = {'curves': nbhood_sharp_curves}
    return nbhood_features



In [62]:
config = {
  "n_patches_per_mesh": 1,
  "neighbourhood": {
    "type": "random_euclidean_sphere",
    "n_vertices": 500,
    "centroid": 0,
    "radius": 10.0,
    "radius_delta": 0.5
  },
  "sampling": {
    "type": "poisson_disk",
    "n_points": 1024,
    "upsampling_factor": 3,
    "poisson_disk_radius": 0.1
  },
  "noise": {
    "type": "no_noise",
  },
  "annotation": {
    "type": "resampling",
    "sharp_discretization": 0.1,
    "distance_upper_bound": 3.2
  }
}

In [63]:
n_patches_per_mesh = config['n_patches_per_mesh']
nbhood_extractor = load_func_from_config(NBHOOD_BY_TYPE, config['neighbourhood'])
sampler = load_func_from_config(SAMPLER_BY_TYPE, config['sampling'])
noiser = load_func_from_config(NOISE_BY_TYPE, config['noise'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])


In [66]:
chunk = ABCChunk(['/home/artonson/tmp/abc/abc_0056_obj_v00.7z', '/home/artonson/tmp/abc/abc_0056_feat_v00.7z'])

In [67]:
item = chunk.get("00560651_308a573e0ac8c0365257e352_001")

In [60]:
mesh = trimesh_load(item.obj)
features = yaml.load(item.feat, Loader=Loader)

In [61]:
point_patches = []

# index the mesh using a neighbourhood functions class
# (this internally may call indexing, so for repeated invocation one passes the mesh)
nbhood_extractor.index(mesh)

for patch_idx in tqdm(range(n_patches_per_mesh)):
    # extract neighbourhood
    nbhood, orig_vert_indices, orig_face_indexes = nbhood_extractor.get_nbhood()

    # sample the neighbourhood to form a point patch
    points, normals = sampler.sample(nbhood)

    # create a noisy sample
    noisy_points = noiser.make_noise(points, normals)

    # create annotations: condition the features onto the nbhood, then compute the TSharpDF
    nbhood_features = compute_curves_nbhood(features, orig_vert_indices, orig_face_indexes)
    distances, directions = annotator.annotate(nbhood, nbhood_features, noisy_points)

    has_sharp = any(curve['sharp'] for curve in nbhood_features['curves'])
    patch_info = {
        'points': noisy_points,
        'distances': distances,
        'directions': directions,
        'item_id': 'item.item_id',
        'orig_vert_indices': orig_vert_indices,
        'orig_face_indexes': orig_face_indexes,
        'has_sharp': has_sharp
    }
    point_patches.append(patch_info)
    
display_sharpness(mesh=mesh, plot_meshvert=False,
                  samples=point_patches[0]['points'], samples_psize=0.4,
                  samples_distances=point_patches[0]['distances'])

  0%|          | 0/1 [00:00<?, ?it/s]/home/artonson/repos/sharp_features/sharpf/data/annotation.py:82: RuntimeWarning: invalid value encountered in true_divide
  directions[~far_from_sharp] /= np.linalg.norm(directions[~far_from_sharp], axis=1, keepdims=True)
100%|██████████| 1/1 [00:33<00:00, 33.04s/it]

3.2



/home/artonson/.pyenv/versions/3.6.6/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/artonson/.pyenv/versions/3.6.6/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [7]:
import h5py
import numpy as np

In [8]:
with h5py.File('/home/artonson/tmp/abc_0030_8016_8183.hdf5', 'r') as f:
    points = np.array(f['points'])
    distances = np.array(f['distances'])
    directions = np.array(f['directions'])

In [9]:
directions.shape

(370, 8192, 3)

In [38]:
idx = np.random.randint(len(points))
d = np.array(distances[idx])
d[d>1.0] = 1.0
display_sharpness(mesh=None, plot_meshvert=False,
                  samples=points[idx], samples_psize=0.2,
                  samples_distances=d)

1.0


/home/artonson/.pyenv/versions/3.6.6/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [39]:
!ls /home/artonson/tmp/ec_net_results

abc_0030_8016_8183_points_0_outputedge.ply
abc_0030_8016_8183_points_0_output.xyz
abc_0030_8016_8183_points_100_outputedge.ply
abc_0030_8016_8183_points_100_output.xyz
abc_0030_8016_8183_points_101_outputedge.ply
abc_0030_8016_8183_points_101_output.xyz
abc_0030_8016_8183_points_102_outputedge.ply
abc_0030_8016_8183_points_102_output.xyz
abc_0030_8016_8183_points_103_outputedge.ply
abc_0030_8016_8183_points_103_output.xyz
abc_0030_8016_8183_points_104_outputedge.ply
abc_0030_8016_8183_points_104_output.xyz
abc_0030_8016_8183_points_105_outputedge.ply
abc_0030_8016_8183_points_105_output.xyz
abc_0030_8016_8183_points_106_outputedge.ply
abc_0030_8016_8183_points_106_output.xyz
abc_0030_8016_8183_points_107_outputedge.ply
abc_0030_8016_8183_points_107_output.xyz
abc_0030_8016_8183_points_108_outputedge.ply
abc_0030_8016_8183_points_108_output.xyz
abc_0030_8016_8183_points_109_outputedge.ply
abc_0030_8016_8183_points_109_output.xyz
abc_0030_8016_8183_points_10_outputedge.ply
abc_0030_8016_

In [40]:
import trimesh

In [43]:
!cat /home/artonson/tmp/ec_net_results/abc_0030_8016_8183_points_349_outputedge.ply

ply
format ascii 1.0
comment VCGLIB generated
element vertex 136
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
property uchar alpha
property float quality
element face 0
property list uchar int vertex_indices
end_header
0.821977 0.000361 -0.557161 95 0 166 255 0.017396
0.817578 -0.000159 -0.552442 101 0 167 255 0.019134
0.820501 -0.000995 -0.544581 103 0 167 255 0.019262
0.828871 -0.001619 -0.547579 103 0 167 255 0.019481
0.828132 -0.004319 -0.549183 106 0 167 255 0.020063
0.819156 -0.002820 -0.553271 111 0 168 255 0.021157
0.824117 -0.004073 -0.537716 111 0 168 255 0.021266
0.836668 0.002363 -0.543558 111 0 168 255 0.021392
0.912680 -0.003257 -0.141407 112 0 168 255 0.021627
0.843787 0.006252 -0.533400 112 0 168 255 0.021643
0.838531 -0.004503 -0.532944 112 0 168 255 0.021730
0.825570 0.003233 -0.562603 112 0 168 255 0.021734
0.830472 0.000446 -0.549746 112 0 168 255 0.021814
0.818112 -0.002618 -0.541888 112 0 168 255 0.

In [45]:
trimesh.load('/home/artonson/tmp/ec_net_results/abc_0030_8016_8183_points_349_outputedge.xyz', file_type='xyz')

ValueError: File type: xyz not supported